In [ ]:
!pip install max --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.0/285.0 MB 5.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/modular/mojo-gpu-puzzles

Cloning into 'mojo-gpu-puzzles'...
remote: Enumerating objects: 4784, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 4784 (delta 316), reused 278 (delta 234), pack-reused 4387 (from 2)
Receiving objects: 100% (4784/4784), 98.15 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (2979/2979), done.


In [ ]:
!curl -fsSL https://astral.sh/uv/install.sh | sh

downloading uv 0.7.13 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [ ]:
import max.support.notebook

In [ ]:
def save_code_to_file(text: str, filename: str):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

In [ ]:
mojo_code = """
from testing import assert_equal
from gpu.host import DeviceContext

# ANCHOR: dot_product_layout_tensor
from gpu import thread_idx, block_idx, block_dim, barrier
from layout import Layout, LayoutTensor
from layout.tensor_builder import LayoutTensorBuild as tb


alias TPB = 8
alias SIZE = 8
alias BLOCKS_PER_GRID = (1, 1)
alias THREADS_PER_BLOCK = (SIZE, 1)
alias dtype = DType.float32
alias layout = Layout.row_major(SIZE)
alias out_layout = Layout.row_major(1)


fn dot_product[
    in_layout: Layout, out_layout: Layout
](
    output: LayoutTensor[mut=True, dtype, out_layout],
    a: LayoutTensor[mut=True, dtype, in_layout],
    b: LayoutTensor[mut=True, dtype, in_layout],
    size: Int,
):
    # FILL ME IN (roughly 13 lines)
    shared = tb[dtype]().row_major[TPB]().shared().alloc()

    global_i = block_dim.x * block_idx.x + thread_idx.x
    local_i = thread_idx.x

    if global_i < size:
      shared[local_i] = a[global_i] * b[global_i]

    barrier()

    stride = TPB // 2
    while stride > 0:
      if local_i < stride:
        shared[local_i] += shared[local_i + stride]
      barrier()
      stride //= 2

    if local_i == 0:
      output[global_i] = shared[local_i]



# ANCHOR_END: dot_product_layout_tensor


def main():
    with DeviceContext() as ctx:
        out = ctx.enqueue_create_buffer[dtype](1).enqueue_fill(0)
        a = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)
        b = ctx.enqueue_create_buffer[dtype](SIZE).enqueue_fill(0)

        with a.map_to_host() as a_host, b.map_to_host() as b_host:
            for i in range(SIZE):
                a_host[i] = i
                b_host[i] = i

        out_tensor = LayoutTensor[dtype, out_layout](out.unsafe_ptr())
        a_tensor = LayoutTensor[dtype, layout](a.unsafe_ptr())
        b_tensor = LayoutTensor[dtype, layout](b.unsafe_ptr())

        ctx.enqueue_function[dot_product[layout, out_layout]](
            out_tensor,
            a_tensor,
            b_tensor,
            SIZE,
            grid_dim=BLOCKS_PER_GRID,
            block_dim=THREADS_PER_BLOCK,
        )

        expected = ctx.enqueue_create_host_buffer[dtype](1).enqueue_fill(0)
        ctx.synchronize()

        with a.map_to_host() as a_host, b.map_to_host() as b_host:
            for i in range(SIZE):
                expected[0] += a_host[i] * b_host[i]

        with out.map_to_host() as out_host:
            print("out:", out_host)
            print("expected:", expected)
            assert_equal(out_host[0], expected[0])

"""

In [ ]:
save_code_to_file(mojo_code, "/content/mojo-gpu-puzzles/problems/p10/p10_layout_tensor.mojo")

In [ ]:
!cd /content/mojo-gpu-puzzles && uv run poe p10_layout_tensor

Poe => mojo problems/p10/p10_layout_tensor.mojo
out: HostBuffer([140.0])
expected: HostBuffer([140.0])
